In [1]:
# IMPORTANT!!!
# Desar la pràctica posant els NIUs dels membres del grup
# És imprescindible que el worksheet es pugui avaluar completament (Cell->Run All) sense que es produeixi cap error de sintaxi. 
# Cal lliurar el worksheet ABANS de finalitzar la vostra sessió de pràctiques.
# No s'avaluaran worksheets que no compleixin els requisits anteriors.

NIU_ESTUDIANT_1 = ""
NOM_ESTUDIANT_1 = ""

NIU_ESTUDIANT_2 = ""
NOM_ESTUDIANT_2 = ""

In [2]:
#########################################################################################
# 
# PUBLIC HELPERS
#
# You can use these functions and definitions in your implementation
#
#########################################################################################

MAX_TARGET = "F"*64

# Get a hex string with the SHA256 sum of the message
def UAB_btc_hash(message):
    import hashlib
    return hashlib.sha256(message.encode('utf-8')).hexdigest()
    
# Convert a hex string to an integer value
def UAB_hexString_to_int(hexString):
    return int(hexString, base=16)
    
# Returns the string concatenation of the integers in list
def UAB_concatenate_ints_as_strings(list):
    return "".join(str(e) for e in list)

#########################################################################################
# 
# DATA STRUCTURES
#
#########################################################################################

# Transaction structure
class transaction_struct():
    
    # Initialize
    def __init__(self, h = None):
        if h:
            self.transaction_hash = h
        else:
            self.transaction_hash = UAB_btc_hash(str(ZZ.random_element()))

    # Show transaction content
    def print_me(self):
        print("Transaction")
        print("  transaction_hash:", self.transaction_hash)
    
    # Serialize transaction structure
    def serialize(self):
        return UAB_concatenate_ints_as_strings([self.transaction_hash])

    # Get transaction hash
    def get_hash(self):
        return self.transaction_hash


# Block header structure                
class block_header_struct():
    
    # Initialize
    def __init__(self):
        self.version = 2
        self.previous_block_hash = None
        self.merkle_root = None
        self.time = None
        self.target = None
        self.nonce = None
        
    # Show block header content
    def print_me(self):
        print("Block header")
        print("  version:", self.version)
        print("  previous_block_hash:", self.previous_block_hash)
        print("  merkle_root:", self.merkle_root)
        print("  time:", self.time)
        print("  target:", self.target)
        print("  nonce:", self.nonce)
        
    # Serialize block header
    def serialize(self):
        s = [ self.version, self.previous_block_hash, self.merkle_root, self.time, self.target, self.nonce]
        return UAB_concatenate_ints_as_strings(s)


# Block structure  
class block_struct():
    
    # Initialize
    def __init__(self):
        self.block_header = None
        self.txs = []
        
    # Show block content
    def print_me(self):
        print("Block")
        print("  block_header:", )
        self.block_header.print_me()
        print("  txs:", [tx.get_hash() for tx in self.txs])

    # Serialize block
    def serialize(self):
        s = [ self.block_header, self.txs]
        return UAB_concatenate_ints_as_strings(s)
     
    # Get block hash   
    def get_hash(self):
        return UAB_btc_hash(self.block_header.serialize())


# Blockchain structure  
class blockchain_struct():
    
    # Initialize
    def __init__(self):
        self.blocks = []
    
    # Show blockchain content
    def print_me(self):
        for block in self.blocks:
            block.print_me()
    
    # Add a new block to the blockchain
    def add_block(self, block):
        self.blocks.append(block)
        
    # Get block list
    def get_blocks(self):
        return self.blocks

In [3]:
# EXERCISE 1a: Compute merkle tree root
#
# Function UAB_compute_merkle_root.
# * Parameter tx_list: list of transactions (transaction_struct objects)
# * Returns: string with the root of the merkle tree
# 
def pairwise(l):
    it = iter(l)
    return zip(it,it)

def concatenate_hash(h):
    h1, h2 = h[0].get_hash(), h[1].get_hash()
    return transaction_struct(UAB_btc_hash(h1+h2))

from math import log2

def UAB_compute_merkle_root(tx_list):
    size = len(tx_list)
    bolean = size & (size-1)
    if size == 0: return UAB_btc_hash("")
    if size == 1: return tx_list[0].get_hash()
    if bolean:
        newSize = 2**(int(log2(size)) + 1)
        tx_list = tx_list + [tx_list[-1] for _ in range(newSize-size)]
    l = [concatenate_hash(tx) for tx in pairwise(tx_list)]
    return UAB_compute_merkle_root(l)

In [4]:
# EXERCISE 1b: Checks if a given transaction is included in a merkle tree
#
# Function UAB_validate_inclusion_simplified.
# * Parameter tx: the transaction to verify (a transaction_struct object)
# * Parameter merkle_root: string with the hash of the merkle root
# * Parameter merkle_path: the merkle path with the hash values needed to verify the tx (a list of tuples)
# * Returns: boolean, True if the transaction is included in the tree, False otherwise
# 

def UAB_validate_inclusion_simplified(tx, merkle_root, merkle_path):
    size = len(merkle_path)
    if size == 0: return merkle_root == tx.get_hash()

    current_hash = tx.get_hash()

    for pos, h in merkle_path:
        concat = [current_hash+h, h+current_hash][pos]
        current_hash = UAB_btc_hash(concat)

    return current_hash == merkle_root

In [5]:
# EXERCISE 1b(2): Check that tx6 belongs to the merkle tree shown in the pdf (using UAB_validate_inclusion_simplified). 
#

tx1 = transaction_struct("c2356069e9d1e79ca924378153cfbbfb4d4416b1f99d41a2940bfdb66c5319db")
tx2 = transaction_struct("4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328cb08b5531fcacdabf8a")
tx3 = transaction_struct("03b26944890929ff751653acb2f2af795cee38f937f379f52ed654a68ce91216")
tx4 = transaction_struct("19581e27de7ced00ff1ce50b2047e7a567c76b1cbaebabe5ef03f7c3017bb5b7")
tx5 = transaction_struct("d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f90da3a666eec13ab35")
tx6 = transaction_struct("5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91b46729d73a27fb57e9")
tx7 = transaction_struct("6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d49c01e52ddb7875b4b")

#### IMPLEMENTATION GOES HERE ####

##################################

In [6]:
# EXERCISE 2: Adds a block to a blockchain
#
# Function UAB_blockchain_mine.
# * Parameter blk_chain: the original blockchain (a blockchain_struct object)
# * Parameter tx_list: a list of transactions to include in the new block (transaction_struct objects)
# * Parameter target: optional, string with the current PoW target (represented in hexadecimal)
# * Returns: the new blockchain (the original blockchain with the new block appended)

from datetime import datetime
import random
import string

def random_string():
    characters = string.ascii_letters + string.digits + string.punctuation
    return ''.join(random.choice(characters) for i in range(random.randint(8, 26)))

def UAB_blockchain_mine(blk_chain, tx_list, target = MAX_TARGET):
    #conseguir el hash anterior
    blocks = blk_chain.get_blocks()
    if len(blocks):
        previous = blocks[-1].get_hash()
    else:
        previous = None

    #construccion del header del bloque
    header = block_header_struct()
    header.previous_block_hash = previous
    header.merkle_root         = UAB_compute_merkle_root(tx_list)
    header.time                = datetime.today()
    header.target              = target

    #construccion del bloque
    block = block_struct()
    block.block_header = header
    block.txs          = tx_list

    #prof of work
    target_int   = UAB_hexString_to_int(target)
    header.nonce = UAB_hexString_to_int(UAB_btc_hash(random_string()))
    work         = UAB_hexString_to_int(block.get_hash())
    while work > target_int:
        header.nonce = UAB_hexString_to_int(UAB_btc_hash(random_string()))
        work         = UAB_hexString_to_int(block.get_hash())

    blk_chain.add_block(block)
    return blk_chain

In [7]:
####################################################################################
# TEST CASES EXERCISE 1a
####################################################################################

tx1 = transaction_struct("1bad6b8cf97131fceab8543e81f7757195fbb1d36b376ee994ad1cf17699c464")
tx2 = transaction_struct("cf3bae39dd692048a8bf961182e6a34dfd323eeb0748e162eaf055107f1cb873")
tx3 = transaction_struct("6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d49c01e52ddb7875b4b")
tx4 = transaction_struct("615bdd17c2556f82f384392ea8557f8cc88b03501c759e23093ab0b2a9b5cd48")
tx5 = transaction_struct("19581e27de7ced00ff1ce50b2047e7a567c76b1cbaebabe5ef03f7c3017bb5b7")
tx6 = transaction_struct("d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f90da3a666eec13ab35")
tx7 = transaction_struct("e5e0093f285a4fb94c3fcc2ad7fd04edd10d429ccda87a9aa5e4718efadf182e")
tx8 = transaction_struct("5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91b46729d73a27fb57e9")
tx9 = transaction_struct("03b26944890929ff751653acb2f2af795cee38f937f379f52ed654a68ce91216")
tx10 = transaction_struct("163f9d874bf45bcce929f64cc69e816219b0f000e374076c1d3efe0a26ca6b6e")

def test_case_1a(name, tx_list, exp_merkle):    
    merkle = UAB_compute_merkle_root(tx_list)
    print("Test", name + ":", merkle == exp_merkle)

exp_merkle = "e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855"
test_case_1a("1a.1", [], exp_merkle) 

exp_merkle = "1bad6b8cf97131fceab8543e81f7757195fbb1d36b376ee994ad1cf17699c464"
test_case_1a("1a.2", [tx1], exp_merkle) 

exp_merkle = "ad3ee4ac443155d71fa2cd251075c50dda10a3991ffea21ea264400ef365312d"
test_case_1a("1a.3", [tx1, tx2], exp_merkle) 

exp_merkle = "1d61fc2b1cb988a0bddd5dc00f942e468c2957f8527a1189b79531b46680d852"
test_case_1a("1a.4", [tx1, tx2, tx3, tx4], exp_merkle) 

exp_merkle = "a1095d369acb94778091ceccbb75719ae5f9941107d1f965174c6aebcb48d631"
test_case_1a("1a.5", [tx1, tx2, tx3, tx4, tx5, tx6, tx7, tx8], exp_merkle)

exp_merkle = "33bbe18031d03aa444e5ce1426d8a992e83210d77af4fec16ef32e779987a317"
test_case_1a("1a.6", [tx1, tx2, tx3], exp_merkle) 

exp_merkle = "9c7c326461af1854e747665a6c1657248c26cef66680301907952cc7aef43c7d"
test_case_1a("1a.7", [tx1, tx2, tx3, tx4, tx5, tx6, tx7], exp_merkle) 

exp_merkle = "0e044c1cf28cb3361dacb8a55275f03963d9a210676e3cae337828751149e494"
test_case_1a("1a.8", [tx1, tx2, tx3, tx4, tx5, tx6], exp_merkle) 

exp_merkle = "9bce55c90dbd3c65086549d556feea84f4b022a6e9d9d29824e3daf237656906"
test_case_1a("1a.9", [tx1, tx2, tx3, tx4, tx5], exp_merkle)

Test 1a.1: True
Test 1a.2: True
Test 1a.3: True
Test 1a.4: True
Test 1a.5: True
Test 1a.6: True
Test 1a.7: True
Test 1a.8: True
Test 1a.9: True


In [8]:
####################################################################################
# TEST CASES EXERCISE 1b
####################################################################################

tx1 = transaction_struct("1bad6b8cf97131fceab8543e81f7757195fbb1d36b376ee994ad1cf17699c464")
tx2 = transaction_struct("cf3bae39dd692048a8bf961182e6a34dfd323eeb0748e162eaf055107f1cb873")
tx3 = transaction_struct("6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d49c01e52ddb7875b4b")
tx4 = transaction_struct("615bdd17c2556f82f384392ea8557f8cc88b03501c759e23093ab0b2a9b5cd48")
tx5 = transaction_struct("19581e27de7ced00ff1ce50b2047e7a567c76b1cbaebabe5ef03f7c3017bb5b7")
tx6 = transaction_struct("d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f90da3a666eec13ab35")
tx7 = transaction_struct("e5e0093f285a4fb94c3fcc2ad7fd04edd10d429ccda87a9aa5e4718efadf182e")
tx8 = transaction_struct("5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91b46729d73a27fb57e9")
tx9 = transaction_struct("03b26944890929ff751653acb2f2af795cee38f937f379f52ed654a68ce91216")
tx10 = transaction_struct("163f9d874bf45bcce929f64cc69e816219b0f000e374076c1d3efe0a26ca6b6e")

def test_case_1b(name, tx, merkle_root, merkle_path, exp_result):    
    r = UAB_validate_inclusion_simplified(tx, merkle_root, merkle_path)
    print("Test", name + ":", r == exp_result)

test_case_1b("1b.1", tx1, "1bad6b8cf97131fceab8543e81f7757195fbb1d36b376ee994ad1cf17699c464", [], True)
test_case_1b("1b.2", tx1, "ad3ee4ac443155d71fa2cd251075c50dda10a3991ffea21ea264400ef365312d", [(0, "cf3bae39dd692048a8bf961182e6a34dfd323eeb0748e162eaf055107f1cb873")], True)
test_case_1b("1b.3", tx2, "ad3ee4ac443155d71fa2cd251075c50dda10a3991ffea21ea264400ef365312d", [(1, "1bad6b8cf97131fceab8543e81f7757195fbb1d36b376ee994ad1cf17699c464")], True)
test_case_1b("1b.4", tx1, "33bbe18031d03aa444e5ce1426d8a992e83210d77af4fec16ef32e779987a317", [(0, "cf3bae39dd692048a8bf961182e6a34dfd323eeb0748e162eaf055107f1cb873"), (0, "3eff7c5314a5ed2d5d8fdad16bbc4851cd98b9861c950854246318c5576a37fd")], True)
test_case_1b("1b.5", tx3, "33bbe18031d03aa444e5ce1426d8a992e83210d77af4fec16ef32e779987a317", [(0, "6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d49c01e52ddb7875b4b"), (1, "ad3ee4ac443155d71fa2cd251075c50dda10a3991ffea21ea264400ef365312d")], True)
test_case_1b("1b.6", tx6, "a1095d369acb94778091ceccbb75719ae5f9941107d1f965174c6aebcb48d631", [(1, "19581e27de7ced00ff1ce50b2047e7a567c76b1cbaebabe5ef03f7c3017bb5b7"), (0, "96511d2d18696af94fc302da346d749fbbd99181c0dbe668a57f2fe92f18d580"), (1, "1d61fc2b1cb988a0bddd5dc00f942e468c2957f8527a1189b79531b46680d852")], True)
test_case_1b("1b.7", tx1, "1bad6b8cf97131fceab8543e81f7757195fbb1d36b376ee994ad1cf17699c474", [], False)
test_case_1b("1b.8", tx1, "ad3ee4ac443155d71fa2cd251075c50dda10a3991ffea21ea264400ef365312d", [(1, "cf3bae39dd692048a8bf961182e6a34dfd323eeb0748e162eaf055107f1cb873")], False)
test_case_1b("1b.9", tx1, "ad3ee4ac443155d71fa2cd251075c50dda10a3991ffea21ea264400ef3653125", [(0, "cf3bae39dd692048a8bf961182e6a34dfd323eeb0748e162eaf055107f1cb873")], False)
test_case_1b("1b.10", tx2, "ad3ee4ac443155d71fa2cd251075c50dda10a3991ffea21ea264400ef365312d", [(0, "1bad6b8cf97131fceab8543e81f7757195fbb1d36b376ee994ad1cf17699c464")], False)
test_case_1b("1b.11", tx2, "ad3ee4ac443155d71fa2cd251075c50dda10a3991ffea21ea264400ef365313d", [(1, "1bad6b8cf97131fceab8543e81f7757195fbb1d36b376ee994ad1cf17699c464")], False)
test_case_1b("1b.12", tx1, "33bbe18031d03aa444e5ce1426d8a992e83210d77af4fec16ef32e779987a317", [(1, "cf3bae39dd692048a8bf961182e6a34dfd323eeb0748e162eaf055107f1cb873"), (0, "3eff7c5314a5ed2d5d8fdad16bbc4851cd98b9861c950854246318c5576a37fd")], False)
test_case_1b("1b.13", tx1, "33bbe18031d03aa444e5ce1426d8a992e83210d77af4fec16ef32e779987a317", [(0, "cf3bae39dd692048a8bf961182e6a34dfd323eeb0748e162eaf055107f1cb874"), (0, "3eff7c5314a5ed2d5d8fdad16bbc4851cd98b9861c950854246318c5576a37fd")], False)
test_case_1b("1b.14", tx1, "33bbe18031d03aa444e5ce1426d8a992e83210d77af4fec16ef32e779987a317", [(0, "cf3bae39dd692048a8bf961182e6a34dfd323eeb0748e162eaf055107f1cb873"), (0, "3eff7c5314a5ed2d5d8fdad16bbc4851cd98b9861c950854246318c5576a370d")], False)
test_case_1b("1b.15", tx1, "33bbe18031d03aa444e5ce1426d8a992e83210d77af4fec16ef32e779987a417", [(0, "cf3bae39dd692048a8bf961182e6a34dfd323eeb0748e162eaf055107f1cb873"), (0, "3eff7c5314a5ed2d5d8fdad16bbc4851cd98b9861c950854246318c5576a37fd")], False)

Test 1b.1: True
Test 1b.2: True
Test 1b.3: True
Test 1b.4: True
Test 1b.5: True
Test 1b.6: True
Test 1b.7: True
Test 1b.8: True
Test 1b.9: True
Test 1b.10: True
Test 1b.11: True
Test 1b.12: True
Test 1b.13: True
Test 1b.14: True
Test 1b.15: True


In [9]:
####################################################################################
# TEST CASES EXERCISE 2
####################################################################################

def test_case_2(name, current_blkchain, tx_list, target, exp_result):
    
    blk_chain = UAB_blockchain_mine(current_blkchain, tx_list, target=target)
    t1 = (len(blk_chain.get_blocks()) == exp_result["num_blocks"])
    blk = blk_chain.get_blocks()[-1]
    t2 = blk.txs == exp_result["tx_list"]
    t3 = blk.block_header.previous_block_hash == exp_result["previous_block_hash"]
    t4 = blk.block_header.merkle_root == exp_result["merkle_root"]
    t5 = blk.block_header.time != None
    t6 = blk.block_header.target == exp_result["target"]    
    t7 = UAB_hexString_to_int( blk.get_hash()) <= UAB_hexString_to_int(target)
    print("Test", name + ":", t1 & t2 & t3 & t4 & t5 & t6 & t7)
    
    return blk_chain
    
try:
    blk_chain_0 = blockchain_struct()
    
    exp_result = {"num_blocks": 1, "tx_list": [], "previous_block_hash": None, "merkle_root": "e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855", "target": MAX_TARGET}
    blk_chain_1 = test_case_2("2.1", blk_chain_0, [], MAX_TARGET, exp_result)
    
    exp_result = {"num_blocks": 2, "tx_list": [tx1], "previous_block_hash": blk_chain_1.get_blocks()[-1].get_hash(), "merkle_root": "1bad6b8cf97131fceab8543e81f7757195fbb1d36b376ee994ad1cf17699c464", "target": MAX_TARGET}
    blk_chain_2 = test_case_2("2.2", blk_chain_1, [tx1], MAX_TARGET, exp_result)
    
    exp_result = {"num_blocks": 3, "tx_list": [tx2, tx3, tx4], "previous_block_hash": blk_chain_2.get_blocks()[-1].get_hash(), "merkle_root": "bc139c7223903cd0e309d58613198787ba25bfd860120bae59472e9af4371520", "target": "00" + "F"*62}
    blk_chain_3 = test_case_2("2.3", blk_chain_2, [tx2, tx3, tx4], "00" + "F"*62, exp_result)
    
    exp_result = {"num_blocks": 4, "tx_list": [tx5], "previous_block_hash": blk_chain_3.get_blocks()[-1].get_hash(), "merkle_root": "19581e27de7ced00ff1ce50b2047e7a567c76b1cbaebabe5ef03f7c3017bb5b7", "target": "00" + "F"*62}
    blk_chain_4 = test_case_2("2.4", blk_chain_3, [tx5], "00" + "F"*62, exp_result)
    
    exp_result = {"num_blocks": 5, "tx_list": [tx6, tx7, tx8, tx9], "previous_block_hash": blk_chain_4.get_blocks()[-1].get_hash(), "merkle_root": "380139b9b93884139aaf8362830e8754f7f59788cfbe874fdb46985bbf45bdf4", "target": "00" + "F"*62}
    blk_chain_5 = test_case_2("2.5", blk_chain_4, [tx6, tx7, tx8, tx9], "00" + "F"*62, exp_result)
    
except Exception as e:
    print("Test", "2X" + ":", False)

Test 2.1: True
Test 2.2: True
Test 2.3: True
Test 2.4: True


Test 2.5: True
